In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [148]:
data = pd.read_csv('tmdb_5000_movies.csv')
data=data[['id','genres','vote_average','vote_count', 'popularity','title','keywords','overview']]

In [149]:
data['vote_count'].quantile(0.9)

1838.4000000000015

In [150]:
tmp_data = data.copy().loc[data['vote_count'] >= 1838]

In [151]:
tmp_data

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...,...
4291,176,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 9648, ""n...",7.2,2184,63.655973,Saw,"[{""id"": 1366, ""name"": ""shotgun""}, {""id"": 13000...",Obsessed with teaching his victims the value o...
4300,500,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 53, ""name...",8.0,3697,66.925866,Reservoir Dogs,"[{""id"": 2052, ""name"": ""traitor""}, {""id"": 6099,...",A botched robbery indicates a police informant...
4302,429,"[{""id"": 37, ""name"": ""Western""}]",8.1,2311,88.377076,"The Good, the Bad and the Ugly","[{""id"": 801, ""name"": ""bounty hunter""}, {""id"": ...",While the Civil War rages between the Union an...
4337,103,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",8.0,2535,58.845025,Taxi Driver,"[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...",A mentally unstable Vietnam War veteran works ...


In [152]:
c = data['vote_average'].mean()
c
m = 1838

In [153]:
def weighted_rating(x, m=m, c=c):
    v=x['vote_count']
    r=x['vote_average']
    return (v / (v+m) * r) + (m / (v+m) * c)
data['score'] = data.apply(weighted_rating, axis=1)

In [154]:
data

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",7.050697
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",6.665732
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,6.239405
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,7.346766
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",6.096368
...,...,...,...,...,...,...,...,...,...
4798,9367,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",6.6,238,14.269792,El Mariachi,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...,6.150391
4799,72766,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",5.9,5,0.642552,Newlyweds,[],A newlywed couple's honeymoon is upended by th...,6.091650
4800,231617,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",7.0,6,1.444476,"Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...",6.095125
4801,126186,[],5.7,7,0.857008,Shanghai Calling,[],When ambitious New York attorney Sam is sent t...,6.090684


In [155]:
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

In [156]:
data['genres']=data['genres'].apply(lambda x:[d['name']for d in x]).apply(lambda x : " ".join(x))
data['keywords']=data['keywords'].apply(lambda x:[d['name']for d in x]).apply(lambda x : " ".join(x))

In [157]:
data

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.050697
1,285,Adventure Fantasy Action,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.665732
2,206647,Action Adventure Crime,6.3,4466,107.376788,Spectre,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...,6.239405
3,49026,Action Crime Drama Thriller,7.6,9106,112.312950,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.346766
4,49529,Action Adventure Science Fiction,6.1,2124,43.926995,John Carter,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",6.096368
...,...,...,...,...,...,...,...,...,...
4798,9367,Action Crime Thriller,6.6,238,14.269792,El Mariachi,united states–mexico barrier legs arms paper k...,El Mariachi just wants to play his guitar and ...,6.150391
4799,72766,Comedy Romance,5.9,5,0.642552,Newlyweds,,A newlywed couple's honeymoon is upended by th...,6.091650
4800,231617,Comedy Drama Romance TV Movie,7.0,6,1.444476,"Signed, Sealed, Delivered",date love at first sight narration investigati...,"""Signed, Sealed, Delivered"" introduces a dedic...",6.095125
4801,126186,,5.7,7,0.857008,Shanghai Calling,,When ambitious New York attorney Sam is sent t...,6.090684


In [65]:
movie_data = pd.read_csv('movies_metadata.csv')
movie_data =  movie_data.loc[movie_data['original_language'] == 'en', :]
movie_data = movie_data[['id', 'title', 'original_language', 'genres']]

C:\Users\jg999\AppData\Local\Temp\ipykernel_24056\1358500802.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_data = pd.read_csv('movies_metadata.csv')


In [66]:
movie_data

,id,title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...
45459,222848,Caged Heat 3000,en,"[{'id': 878, 'name': 'Science Fiction'}]"
45460,30840,Robin Hood,en,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name..."
45463,67758,Betrayal,en,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,Satan Triumphant,en,[]


In [67]:
moive_keyword = pd.read_csv('keywords.csv')
moive_keyword

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [68]:
movie_data.id=movie_data.id.astype(int)

In [69]:
movie_data = pd.merge(movie_data, moive_keyword, on='id')

In [70]:
movie_data['genres']=movie_data['genres'].apply(literal_eval)
movie_data['keywords']=movie_data['keywords'].apply(literal_eval)

In [71]:
movie_data['genres']=movie_data['genres'].apply(lambda x:[d['name']for d in x]).apply(lambda x : " ".join(x))
movie_data['keywords']=movie_data['keywords'].apply(lambda x:[d['name']for d in x]).apply(lambda x : " ".join(x))

In [73]:
movie_data

,id,title,original_language,genres,keywords
0,862,Toy Story,en,Animation Comedy Family,jealousy toy boy friendship friends rivalry bo...
1,8844,Jumanji,en,Adventure Fantasy Family,board game disappearance based on children's b...
2,15602,Grumpier Old Men,en,Romance Comedy,fishing best friend duringcreditsstinger old men
3,31357,Waiting to Exhale,en,Comedy Drama Romance,based on novel interracial relationship single...
4,11862,Father of the Bride Part II,en,Comedy,baby midlife crisis confidence aging daughter ...
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,Science Fiction,
32848,30840,Robin Hood,en,Drama Action Romance,
32849,67758,Betrayal,en,Action Drama Thriller,
32850,227506,Satan Triumphant,en,,


In [74]:
data

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.050697
1,285,Adventure Fantasy Action,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.665732
2,206647,Action Adventure Crime,6.3,4466,107.376788,Spectre,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...,6.239405
3,49026,Action Crime Drama Thriller,7.6,9106,112.312950,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.346766
4,49529,Action Adventure Science Fiction,6.1,2124,43.926995,John Carter,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",6.096368
...,...,...,...,...,...,...,...,...,...
4798,9367,Action Crime Thriller,6.6,238,14.269792,El Mariachi,united states–mexico barrier legs arms paper k...,El Mariachi just wants to play his guitar and ...,6.150391
4799,72766,Comedy Romance,5.9,5,0.642552,Newlyweds,,A newlywed couple's honeymoon is upended by th...,6.091650
4800,231617,Comedy Drama Romance TV Movie,7.0,6,1.444476,"Signed, Sealed, Delivered",date love at first sight narration investigati...,"""Signed, Sealed, Delivered"" introduces a dedic...",6.095125
4801,126186,,5.7,7,0.857008,Shanghai Calling,,When ambitious New York attorney Sam is sent t...,6.090684


In [158]:
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
]

In [76]:
vocab = list(set(w for doc in docs for w in doc.split()))

In [80]:
vocab.sort()
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [159]:
def tf(t, d):
    return d.count(t)
    
def idf(t):    
    df=0 #단어가 등장한 문서의 개수를 저장
    for doc in docs:
        df += t in doc
    return log(n/(df+1)) # idf값이 크다 df값이 작다
def tfidf(t, d):
    return tf(t, d) * idf(t)
# tfidf가 크다 = tf도 크고 idf도 크다, 
# tf가 클라면 특정 단어가 문서 내에서 많이 등장해야댐 (해당 문서 내 빈도^)
# idf가 클라면 전체 문서 내에서 흔하지 않은 단어 (전체 문서 내 희귀도^)

In [78]:
n = len(docs)

In [85]:
res=[]
for i in range(n):
    res.append([]) #[[],[],[],[]]
    d=docs[i] #'먹고 싶은 사과'
    for j in  range(len(vocab)): #['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']
        t=vocab[j]
        res[-1].append(tf(t,d)) #tf('과일이', '먹고 싶은 사과')

In [141]:
res

[[0, 0, 0, 1, 0, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 1, 0, 2, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 1]]

In [91]:
tf_ = pd.DataFrame(res, columns=vocab)# tf == dtm

In [92]:
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [98]:
idf('바나나')

0.28768207245178085

In [101]:
res2 = []
for j in range(len(vocab)):
    t = vocab[j]
    res2.append(idf(t))
res2
pd.DataFrame(res2, index=vocab, columns=['IDF'])

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [102]:
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [104]:
result = []
for i in range(n):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


In [105]:
cosine_similarity(tfidf_, tfidf_) # 코사인 유사도 탐색

array([[1.        , 0.41330849, 0.        , 0.        ],
       [0.41330849, 1.        , 0.29225323, 0.        ],
       [0.        , 0.29225323, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [160]:
data

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.050697
1,285,Adventure Fantasy Action,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.665732
2,206647,Action Adventure Crime,6.3,4466,107.376788,Spectre,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...,6.239405
3,49026,Action Crime Drama Thriller,7.6,9106,112.312950,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.346766
4,49529,Action Adventure Science Fiction,6.1,2124,43.926995,John Carter,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",6.096368
...,...,...,...,...,...,...,...,...,...
4798,9367,Action Crime Thriller,6.6,238,14.269792,El Mariachi,united states–mexico barrier legs arms paper k...,El Mariachi just wants to play his guitar and ...,6.150391
4799,72766,Comedy Romance,5.9,5,0.642552,Newlyweds,,A newlywed couple's honeymoon is upended by th...,6.091650
4800,231617,Comedy Drama Romance TV Movie,7.0,6,1.444476,"Signed, Sealed, Delivered",date love at first sight narration investigati...,"""Signed, Sealed, Delivered"" introduces a dedic...",6.095125
4801,126186,,5.7,7,0.857008,Shanghai Calling,,When ambitious New York attorney Sam is sent t...,6.090684


In [181]:
genres = data['genres'].map(lambda x: x.split())

In [182]:
vocab = list(set(i for g in genres for i in g))

In [183]:
vocab.sort()

In [234]:
vocab

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Fiction',
 'Foreign',
 'History',
 'Horror',
 'Movie',
 'Music',
 'Mystery',
 'Romance',
 'Science',
 'TV',
 'Thriller',
 'War',
 'Western']

In [184]:
n = len(data)

In [185]:
res=[]
for i in genres:
    res.append([]) #[[],[],[],[]]
    for t in  vocab:
        res[-1].append(tf(t,i))

In [186]:
pd.DataFrame(res, index=data['title'], columns=vocab)

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Fiction,...,Horror,Movie,Music,Mystery,Romance,Science,TV,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,,
Avatar,1,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,0,0
Pirates of the Caribbean: At World's End,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Spectre,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Dark Knight Rises,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
John Carter,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
El Mariachi,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Newlyweds,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
"Signed, Sealed, Delivered",0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,1,0,1,0,0,0


In [201]:
idf('Movie')

6.279771424328605

In [203]:
res2 = []
for i in vocab:
    res2.append(idf(i))
res2
pd.DataFrame(res2, index=vocab, columns=['IDF'])

,IDF
Action,1.425140
Adventure,1.803698
Animation,3.017410
Comedy,1.025174
Crime,1.930211
Documentary,3.767466
Drama,0.737202
Family,2.234773
Fantasy,2.424907
Fiction,2.192862


In [206]:
result = []
for i in range(n):
    result.append([])
    d = genres[i]
    for t in vocab:
        result[-1].append(tfidf(t,d))

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Fiction,...,Horror,Movie,Music,Mystery,Romance,Science,TV,Thriller,War,Western
0,1.42514,1.803698,0.0,0.000000,0.000000,0.000000,0.000000,0.0,2.424907,2.192862,...,0.0,0.000000,0.0,0.0,0.000000,2.192862,0.000000,0.000000,0.0,0.0
1,1.42514,1.803698,0.0,0.000000,0.000000,0.000000,0.000000,0.0,2.424907,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,1.42514,1.803698,0.0,0.000000,1.930211,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,1.42514,0.000000,0.0,0.000000,1.930211,0.000000,0.737202,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.326295,0.0,0.0
4,1.42514,1.803698,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.192862,...,0.0,0.000000,0.0,0.0,0.000000,2.192862,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,1.42514,0.000000,0.0,0.000000,1.930211,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.326295,0.0,0.0
4799,0.00000,0.000000,0.0,1.025174,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,1.680172,0.000000,0.000000,0.000000,0.0,0.0
4800,0.00000,0.000000,0.0,1.025174,0.000000,0.000000,0.737202,0.0,0.000000,0.000000,...,0.0,6.279771,0.0,0.0,1.680172,0.000000,6.279771,0.000000,0.0,0.0
4801,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [235]:
tfidf_ = pd.DataFrame(result, columns = vocab, index=data['title'])
tfidf_.loc['Avatar']
tfidf_

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Fiction,...,Horror,Movie,Music,Mystery,Romance,Science,TV,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,,
Avatar,1.42514,1.803698,0.0,0.000000,0.000000,0.000000,0.000000,0.0,2.424907,2.192862,...,0.0,0.000000,0.0,0.0,0.000000,2.192862,0.000000,0.000000,0.0,0.0
Pirates of the Caribbean: At World's End,1.42514,1.803698,0.0,0.000000,0.000000,0.000000,0.000000,0.0,2.424907,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
Spectre,1.42514,1.803698,0.0,0.000000,1.930211,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
The Dark Knight Rises,1.42514,0.000000,0.0,0.000000,1.930211,0.000000,0.737202,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.326295,0.0,0.0
John Carter,1.42514,1.803698,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.192862,...,0.0,0.000000,0.0,0.0,0.000000,2.192862,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
El Mariachi,1.42514,0.000000,0.0,0.000000,1.930211,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.326295,0.0,0.0
Newlyweds,0.00000,0.000000,0.0,1.025174,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,1.680172,0.000000,0.000000,0.000000,0.0,0.0
"Signed, Sealed, Delivered",0.00000,0.000000,0.0,1.025174,0.000000,0.000000,0.737202,0.0,0.000000,0.000000,...,0.0,6.279771,0.0,0.0,1.680172,0.000000,6.279771,0.000000,0.0,0.0


In [224]:
similar = pd.DataFrame(cosine_similarity(tfidf_, tfidf_), index=data['title'], columns=data['title']) # 코사인 유사도 탐색

In [233]:
similar.loc['Avatar'].sort_values(ascending=False).head(10)

title
Avatar                                       1.000000
X-Men: Days of Future Past                   1.000000
Beastmaster 2: Through the Portal of Time    1.000000
Man of Steel                                 1.000000
Superman Returns                             1.000000
Superman II                                  1.000000
Superman                                     1.000000
Jupiter Ascending                            1.000000
The Wolverine                                1.000000
Teenage Mutant Ninja Turtles                 0.975634
Name: Avatar, dtype: float64

In [191]:
def tf(t, d):
    return d.count(t)
    
def idf(t):    
    df=0 #단어가 등장한 문서의 개수를 저장
    for g in genres:
        df += t in g
    return log(n/(df+1)) # idf값이 크다 df값이 작다
def tfidf(t, d):
    return tf(t, d) * idf(t)
# tfidf가 크다 = tf도 크고 idf도 크다, 
# tf가 클라면 특정 단어가 문서 내에서 많이 등장해야댐 (해당 문서 내 빈도^)
# idf가 클라면 전체 문서 내에서 흔하지 않은 단어 (전체 문서 내 희귀도^)